In [1]:
import sys
sys.path.append('../src')

import network as nw
import torch
import generate_kvadraturen as gk
import bus
import road as rd
import traffic_lights as tl
import junction as jn
import numpy as np
import loading_json as load
import json

In [2]:
f = open("../kvadraturen_networks/1_1.json")
data = json.load(f)
f.close()
T = data["T"]
T = 30
N = data["N"]
speed_limits = data["speed_limits"] # Nested list
control_points = data["control_points"] # Nested list
cycle_times = data["cycle_times"] # Nested list

In [5]:
bus_network = gk.generate_kvadraturen_roundabout_w_params(T, N, speed_limits, control_points, cycle_times,
                                                          track_grad=False)

Strandgate to HW cycle: [tensor(50.), tensor(100.)]
Strandgate to tollbod cycle: [tensor(50.), tensor(100.)]
HW cycle: [tensor(80.), tensor(80.)]
Tollbod to festning cycle: [tensor(50.), tensor(100.)]
Dronning to festning cycle [tensor(60.), tensor(120.)]
Dronning to elvegate cycle [tensor(60.), tensor(120.)]


In [11]:
T = 30
bus_network = gk.generate_kvadraturen_w_bus(T, track_grad=False)

Strandgate to HW cycle: [tensor(50.), tensor(100.)]
Strandgate to tollbod cycle: [tensor(50.), tensor(100.)]
HW cycle: [tensor(100.), tensor(50.)]
Tollbod to festning cycle: [tensor(50.), tensor(100.)]
Dronning to festning cycle [tensor(100.), tensor(50.)]
Dronning to elvegate cycle [tensor(50.), tensor(100.)]


In [12]:
# with torch.autograd.set_detect_anomaly(True):
#     densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()

densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()

Bus 2 reached bus stop 0 at time 27.10001564025879, should wait for 29.56817054748535 seconds


In [13]:
objective_fnc = torch.tensor(0.0)
for i in range(len(bus_delays)):
    for j in range(len(bus_delays[i])):
        objective_fnc = objective_fnc + bus_delays[i][j]
print(objective_fnc)

tensor(23.1182)


In [5]:
objective_fnc.backward()
speed_limits = bus_network.get_speed_limit_grads()
traffic_lights = bus_network.get_traffic_light_grads()

In [ ]:
59.24377647  80.          80.          65.87917085  57.49176426
  79.26315085  80.          50.13192135  28.42084207  49.62954308
  99.99005596  50.2654001   99.99631058  50.03542892  99.99824851
  50.29059791  99.99172916  49.99478611  99.99989917  70.12932961
  69.99999647  79.33517426  79.80488551  49.68705362  99.98898422
  59.79298455 119.88902983  60.76570415 119.98281134  59.87032718
 119.99935821  79.77154544  99.98828991  79.96011442  98.09888928

In [9]:
speed_limits[4]

0.04000745341181755

In [5]:
times = list(densities[0].keys())
objective_fnc += densities[0][times[-1]][4]

In [5]:
objective_fnc.backward()

In [6]:
bus_network.get_speed_limit_grads()

[tensor(0.0009),
 tensor(3.3250e-25),
 tensor(-1.3810),
 tensor(-0.6805),
 tensor(0.0400),
 tensor(0.0084),
 tensor(-0.6718),
 tensor(1.4207e-20),
 tensor(-1.2328e-09)]

In [7]:
bus_network.get_traffic_light_grads()

[tensor(-2.7601e-24),
 tensor(0.),
 tensor(-4.7805e-21),
 tensor(0.),
 tensor(-1.5719e-20),
 tensor(0.),
 tensor(-4.5082e-20),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(-4.8610e-20),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(1.1688e-22),
 tensor(0.),
 tensor(3.1627e-27),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(7.5706e-21),
 tensor(0.),
 tensor(-9.7813e-19),
 tensor(0.),
 tensor(-3.1493e-18),
 tensor(0.)]

#### Testing speed of bus coming into junction

In [21]:
# Interesting bus: first bus
test_bus = bus_network.busses[0]
test_bus_lengths = bus_lengths[0]

# Interesting roads:
incoming = bus_network.roads[28]
outgoing = bus_network.roads[14]
in_densities = densities[28]
out_densities = densities[14]

# Interesting junction:
jnc = bus_network.junctions[7]

# Interesting time period: 
start_time = torch.tensor(344)
end_time = torch.tensor(366)

In [29]:
# First check densities on roads in the interesting interval:
times = list(in_densities.keys())
for i, t in enumerate(times):
    if start_time < t < end_time:
        next_t = times[i+1]
        dt = next_t - T
        tot_length = test_bus_lengths[t]
        _, length = test_bus.get_road_id_at_length(tot_length)
        new_length = length / incoming.L
        prev, next, pos = incoming.get_node_at_length(new_length)

        print(f"t = {t}")
        print(f"Bus has travelled {tot_length} meters, {length} meters on current road")
        print(f"This is equivalent to internal nodes {prev}, and {next}, with pos {pos}")
        print(f"Incoming densities: ")
        print(in_densities[t])
        print(f"Outgoing densities: ")
        print(out_densities[t])
        print()

t = 344.200927734375
Bus has travelled 644.899169921875 meters, 94.899169921875 meters on current road
This is equivalent to internal nodes 7.0, and 8.0, with pos 7.091933727264404
Incoming densities: 
tensor([2.3842e-07, 2.3842e-07, 2.3842e-07, 2.3842e-07, 2.3859e-07, 2.3994e-07,
        2.4461e-07, 2.5648e-07, 2.6570e-07, 4.3250e-06, 6.9844e-03, 3.0192e-01,
        9.2697e-01, 9.9957e-01, 1.0000e+00, 1.0000e+00], grad_fn=<CopySlices>)
Outgoing densities: 
tensor([1.3129e-10, 1.3129e-10, 4.7617e-11, 1.5330e-11, 4.4689e-12, 1.3288e-12,
        3.9349e-13, 1.1662e-13, 3.4558e-14, 1.0241e-14, 3.0348e-15, 8.9931e-16,
        2.6650e-16, 7.8972e-17, 2.3402e-17, 6.9349e-18, 2.0550e-18, 6.0904e-19,
        1.8058e-19, 5.3407e-20, 1.5374e-20, 4.5237e-21, 2.8527e-21, 5.1930e-21,
        2.0519e-20, 9.4007e-20, 6.3478e-19, 6.3478e-19], grad_fn=<CopySlices>)

t = 344.65093994140625
Bus has travelled 651.149169921875 meters, 101.149169921875 meters on current road
This is equivalent to internal n

In [34]:
# Check when bus
first_bus_lengths = bus_lengths['0']
first_bus = bus_network.busses[0]
times = first_bus_lengths.keys()
for t in times:
    length = first_bus_lengths[t]
    if length > 700:
        print(f"t = {t}, length = {length}")
        print(f"Road id: {first_bus.get_road_id_at_length(length)}")
        break

t = 365.0, length = 700.0109252929688
Road id: ('tollbod_2fw', 0.01092529296875)


In [4]:
# 27.550016403198242, should wait for 29.835927963256836 seconds
# 344.65093994140625 - 365 - Check calculated speeds...

In [4]:
print(bus_delays)

{0: [tensor(202.3306, grad_fn=<AddBackward0>), tensor(199.9107, grad_fn=<AddBackward0>), tensor(242.2030, grad_fn=<AddBackward0>), tensor(205.3318, grad_fn=<AddBackward0>)], 1: [tensor(23.8124, grad_fn=<AddBackward0>), tensor(5.1012, grad_fn=<AddBackward0>), tensor(0., grad_fn=<AddBackward0>), tensor(119.0426, grad_fn=<AddBackward0>)], 2: [tensor(0.), tensor(0.), tensor(0.), tensor(0.)], 3: [tensor(0., grad_fn=<AddBackward0>), tensor(0., grad_fn=<AddBackward0>), tensor(0., grad_fn=<AddBackward0>), tensor(0.)]}


In [5]:
objective_fnc = torch.tensor(0.0)
for i in range(len(bus_delays)):
    for j in range(len(bus_delays[i])):
        objective_fnc = objective_fnc + bus_delays[i][j]
print(objective_fnc)

tensor(997.7323, grad_fn=<AddBackward0>)


In [6]:
objective_fnc.backward()

In [ ]:
adgHSKKL

NameError: name 'adgHSKKL' is not defined

In [ ]:
rho = torch.tensor([0.1, 0.1, 0.2, 0.45, 0.35, 0.234, 0.124, 0.64, 0.24])
slowdown = torch.tensor([0.1, 0.5, 0.7, 0.2, 0.6, 0.1])
pad = 2
n_internal = len(rho) - 2*pad
for i in range(n_internal+1):
    print(rho[-pad-i], rho[-pad-i-1])

internal = torch.flip(rho[pad-1:-pad+1],dims=[0])
avg = (internal[1:] + internal[:-1]) / 2
torch.ones(6) - (1. - slowdown) * (1. - avg) / 2

tensor(0.6400) tensor(0.1240)
tensor(0.1240) tensor(0.2340)
tensor(0.2340) tensor(0.3500)
tensor(0.3500) tensor(0.4500)
tensor(0.4500) tensor(0.2000)
tensor(0.2000) tensor(0.1000)


tensor([0.7219, 0.7947, 0.8938, 0.7600, 0.8650, 0.6175])

In [12]:
n = 4
m = 3
A = torch.ones((n, m))
B = torch.tensor([[1.3, 5.2, 12.4],
                  [24.1, 0.04, 0.5],
                  [12.1, 12.4, 22.2],
                  [1.2, 4.6, 7.8]])
i = 3
j = 1
tot_sum = torch.tensor(0.0)
mask = torch.ones(n, dtype=torch.bool)
mask[i] = False
filtered_tensor = torch.cat((B[:i,j], B[i+1:,j]))
for l in range(n):
    if l != i:
        tot_sum += B[l,j]
second_sum = torch.sum(B[mask, j])

tot_sum
second_sum - tot_sum

tensor(0.)

In [5]:
densities_loaded = load.convert_from_tensor(densities)
queues_loaded = load.convert_from_tensor(queues)
bus_lengths_loaded = load.convert_from_tensor(bus_lengths)

In [6]:
bus_delays_dict = {0: [float(l.detach()) for l in bus_delays[0]],
                   1: [float(l.detach()) for l in bus_delays[1]],
                   2: [float(l.detach()) for l in bus_delays[2]]}
                #    3: [float(l.detach()) for l in bus_delays[3]]}
                   

In [7]:
# Write above dictionaries to json file
import json
# with open("kvadraturen_results_minimal_1000_variable.json", 'w') as fd:
#     fd.write(json.dumps([densities, queues, bus_lengths, bus_delays_dict]))
with open("kvadraturen_500_orig.json", 'w') as fd:
    fd.write(json.dumps([densities_loaded, queues_loaded, bus_lengths_loaded, bus_delays_dict]))

In [9]:
import json
print("Loading results...")
f = open("kvadraturen_roundabout_750_new.json")
data = json.load(f)
f.close()
densities = data[0]
queues = data[1]
bus_lengths = data[2]
bus_delays = data[3]

Loading results...


In [ ]:
import openGLUtilsGif as opgl
opgl.draw_busses_w_densities(bus_network, [bus_fw, bus_bw, bus_fw_2, bus_bw_2], bus_lengths,
                                    densities, output_name="test_roundabout.gif")

Maximum and minimum x points: 7, -0.5
Maximum and minimum y points: -0.4, 8.9
Window created


: 

In [4]:
import json
print("Loading results...")
f = open("../results/kvadraturen_roundabout_750_new.json")
data = json.load(f)
f.close()
densities = data[0]
queues = data[1]
bus_lengths = data[2]
bus_delays = data[3]

Loading results...


In [10]:
shortened_densities = {}
for road_id in densities.keys():
    shortened_densities[road_id] = {}
    for t in densities[road_id].keys():
        old_dens = densities[road_id][t]
        new_dens = [old_dens[i] for i in range(2, len(old_dens)-2)]
        shortened_densities[road_id][t] = new_dens        

In [11]:
with open("kvadraturen_roundabout_750_internal_new.json", 'w') as fd:
    fd.write(json.dumps([shortened_densities, queues, bus_lengths, bus_delays]))

In [43]:
def short_stick_prob(x, d):
    '''
    Probability of a point x in [0,1] being hit by a stick of length d <= 0.5
    '''
    if x <= d:
        return x / (1 - d)
    elif x <= 1 - d:
        return d / (1 - d)
    else:
        return (1 - x) / (1 - d)
    
def long_stick_prob(x, d):
    '''
    Probability of a point x in [0,1] being hit by a stick of length d > 0.5
    '''
    if x <= 1 - d:
        return x / (1 - d)
    elif x <= d:
        return torch.tensor(1.0)
    else:
        return (1 - x) / (1 - d)
    
def stick_prob(x,d):
    '''
    Probability of a point x in [0,1] being hit by a stick of length d (> 0)
    '''
    if d <= 0.5:
        return short_stick_prob(x,d)
    else:
        return long_stick_prob(x,d)

In [23]:
def trapezoidal_rule(f, a=0.0, b=1.0, n=10):
    '''
    Assuming equidistant spacing
    '''
    dx = (b-a) / n
    xk = torch.linspace(a, b, n+1)
    int = torch.tensor(0.0)
    int += (f(xk[0]) + f(xk[-1]))
    for i in range(1,n):
        int += 2 * f(xk[i])
    int = int * dx / 2
    return int

In [49]:
d_list = [torch.tensor(0.3, requires_grad=True), torch.tensor(0.2, requires_grad=True), torch.tensor(0.7, requires_grad=True)]


# output = trapezoidal_rule(lambda x: 1 - torch.prod(torch.tensor([1-stick_prob(x,d) for d in d_list])), 0, 1, 4)
def temp_func(x):
    integrand = torch.tensor(1.0)
    for d in d_list:
        integrand *= (1-stick_prob(x,d))
    return 1.0 - integrand
# output = trapezoidal_rule(lambda x: 1 - (1-stick_prob(x,d_list[0]))*(1-stick_prob(x,d_list[1]))*(1-stick_prob(x,d_list[2])) , 0, 1, 4)
# output = trapezoidal_rule(temp_func, 0, 1, 4)
# output = trapezoidal_rule(lambda x: 1.0 - torch.prod(torch.stack([(1 - stick_prob(x, d)) for d in d_list])), 0, 1, 4)

# print(output)
temp_func_lambda = lambda x : 1.0 - torch.prod(torch.stack([(1.0 - stick_prob(x, d)) for d in d_list]))
output = trapezoidal_rule(temp_func_lambda, 0, 1, 4)
output

tensor(0.7098, grad_fn=<DivBackward0>)

In [45]:
x = 0.3
print(1.0 - stick_prob(x, d_list[0]), 1.0 - stick_prob(x, d_list[1]), 1.0 - stick_prob(x, d_list[2]))
print(torch.stack([(1.0 - stick_prob(x, d)) for d in d_list]))
print(torch.prod(torch.stack([(1.0 - stick_prob(x, d)) for d in d_list])))


tensor(0.5714, grad_fn=<RsubBackward1>) tensor(0.7500, grad_fn=<RsubBackward1>) tensor(0., grad_fn=<RsubBackward1>)
tensor([0.5714, 0.7500, 0.0000], grad_fn=<StackBackward0>)
tensor(0., grad_fn=<ProdBackward0>)


In [48]:
output.backward()

for d in d_list:
    print(d.grad)

tensor(0.0319)
tensor(0.0837)
tensor(0.6696)


In [ ]:
tensor(0.0319)
tensor(0.0837)
tensor(0.6696)